In [1]:
from pyspark import SparkConf, SparkContext
from collections import OrderedDict
conf = SparkConf().setMaster("local").setAppName("MovieLens")
sc = SparkContext(conf=conf)

24/12/04 15:41:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import os
directory = os.path.join(os.getcwd(), "data") 
filename  = "u.data"
file_path = os.path.join(directory, filename)

### columns
- user_id
- movie_id
- rating
- timestamp

In [3]:
lines = sc.textFile("file:///" + file_path.replace("\\", "/"))

In [15]:
lines.first()

'196\t242\t3\t881250949'

In [4]:
lines.collect()[:5]

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [19]:
# (movie_id, rating)
movie_ratings = lines.map(lambda x: (x.split('\t')[1], int(x.split('\t')[2])))

# 평점 3점 이상인 영화
filtered_movie_ratings = movie_ratings.filter(lambda x: x[1] >= 3)

# 영화별로 그룹화 후 평균 평점
movie_avg_ratings = filtered_movie_ratings \
    .groupByKey() \
    .mapValues(lambda ratings: sum(ratings) / len(ratings))

# 가장 높은 평균 평점을 가진 영화
highest_rated_movie = movie_avg_ratings.max(key=lambda x: x[1])

print(f"Highest Rated Movie ID: {highest_rated_movie[0]}")
print(f"Average Rating: {highest_rated_movie[1]}")

Highest Rated Movie ID: 1306
Average Rating: 5.0
